# RAG with Ollama


> Note: Before proceeding further you need to download and run Ollama: https://ollama.com/.

This notebook demonstrates how to use the `rag` model with Ollama. 
Source: https://dev.to/mohsin_rashid_13537f11a91/rag-with-ollama-1049

* Import Libraries

In [2]:
from langchain_ollama import OllamaLLM
from langchain_ollama.embeddings import OllamaEmbeddings

#from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain

* Loading The LLM (Language Model)

In [3]:
llm = OllamaLLM(model="llama3.2", base_url="http://127.0.0.1:11434")

* Setting Ollama Embeddings

In [4]:
embed_model = OllamaEmbeddings(
    model="llama3.2",
    base_url='http://127.0.0.1:11434'
)

* Loading Text

In [5]:
#text = """
#    In the lush canopy of a tropical rainforest, two mischievous monkeys, Coco and Mango, swung from branch to branch, their playful antics echoing through the trees. They were inseparable companions, sharing everything from juicy fruits to secret hideouts high above the forest floor. One day, while exploring a new part of the forest, Coco stumbled upon a beautiful orchid hidden among the foliage. Entranced by its delicate petals, Coco plucked it and presented it to Mango with a wide grin. Overwhelmed by Coco's gesture of friendship, Mango hugged Coco tightly, cherishing the bond they shared. From that day on, Coco and Mango ventured through the forest together, their friendship growing stronger with each passing adventure. As they watched the sun dip below the horizon, casting a golden glow over the treetops, they knew that no matter what challenges lay ahead, they would always have each other, and their hearts brimmed with joy.
#    """

* Splitting Text into Chunks

In [6]:
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
#chunks = text_splitter.split_text(text)

In [7]:
chunks = ["ILLESCAS, (Géog.) petite ville d'Espagne, dans la nouvelle Castille, à six lieues au sud de Madrid.", 
          "MULHAUSEN, (Géog.) ville impériale d'Allemagne, dans la Thuringe, sous la protection de l'électeur de Saxe, ce qui fait qu'elle est rangée parmi les villes de basse-Saxe ; elle a essuyé bien des calamités en divers tems. Henri le Lion la prit d'assaut en 1181, & la brûla. En 1366 un tremblement de terre en renversa la plus grande partie ; en 1442 un incendie ne lui fut guère moins funeste ; en 1525 elle fut assiegée par l'électeur de Saxe & le landgrave de Hesse, à cause des paysans révoltés qui s'en étoient emparés ; enfin après la paix de Westphalie, les divers partis l'ont ravagée tour-à-tour. Elle est située dans un pays fertile, sur la riviere d'Unstruth, à 5 milles de Nordhausen, 6 N. E. d'Eysenach, 10 N. O. d'Erford, 14 S. O. de Cassel. Long. 28. 14. lat. 51. 13. (D. J.)"
            ]

In [8]:
len(chunks)

2

* Creating a Vector Store (Chroma) from Text

In [9]:
vector_store = Chroma.from_texts(chunks, embed_model)

* Creating a Retriever

In [10]:
retriever = vector_store.as_retriever()

* Creating a Retrieval Chain

In [11]:
chain = create_retrieval_chain(combine_docs_chain=llm,retriever=retriever)

* Retrieval-QA Chat Prompt

In [12]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

/opt/homebrew/Caskroom/miniforge/base/envs/rag-py311/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


* Combining Documents

In [13]:
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

* Final Retrieval Chain

In [14]:
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)    

* Invoking the Retrieval Chain

In [16]:
response = retrieval_chain.invoke({"input": "Dans quel pays se situe Mulhausen ?"})
print(response['answer'])

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


Mulhausen se trouve en Allemagne.


In [17]:
response = retrieval_chain.invoke({"input": "Quel est la localisation de Mulhausen (latitude / longitude) ?"})
print(response['answer'])

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


Mulhouse est située à 51°13' de latitude nord et 28°14' de longitude est.
